# Clasificación

Los clasificadores empleados son LDA y KNN.

In [0]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

Carga del dataset iris

In [0]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

División en conjuntos de entrenamiento y pruebas.

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Inicialización de los clasificadores

In [0]:
lda = LinearDiscriminantAnalysis()
knn = KNeighborsClassifier()

Entrenamiento de los modelos

In [5]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [6]:
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

Predicción

In [0]:
y_pred_knn = knn.predict(X_test)
y_pred_lda = lda.predict(X_test)

# Rendimiento individual

In [8]:
from sklearn.metrics import classification_report

knn_acc = knn.score(X_test, y_test)
lda_acc = lda.score(X_test, y_test)

print(classification_report(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_lda))

print('Precisión del clasificador de K vecinos más cercanos: %.2f%%' % (knn_acc*100))
print('Precisión del clasificador de Análisis discriminante lineal: %.2f%%' % (lda_acc*100))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       1.00      0.75      0.86        16
           2       0.75      1.00      0.86        12

    accuracy                           0.91        45
   macro avg       0.92      0.92      0.90        45
weighted avg       0.93      0.91      0.91        45

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       1.00      0.88      0.93        16
           2       0.86      1.00      0.92        12

    accuracy                           0.96        45
   macro avg       0.95      0.96      0.95        45
weighted avg       0.96      0.96      0.96        45

Precisión del clasificador de K vecinos más cercanos: 91.11%
Precisión del clasificador de Análisis discriminante lineal: 95.56%


# Validación cruzada de 10 iteraciones

In [9]:
from sklearn import model_selection

k_fold_val = model_selection.KFold(n_splits=10, shuffle=True)
result_knn = model_selection.cross_val_score(knn, X, y, cv=k_fold_val, scoring='accuracy')
result_lda = model_selection.cross_val_score(lda, X, y, cv=k_fold_val, scoring='accuracy')

print("KNN: %f (%f)"% (result_knn.mean(), result_knn.std()))
print("LDA: %f (%f)"% (result_lda.mean(), result_lda.std()))


KNN: 0.966667 (0.044721)
LDA: 0.980000 (0.030551)


# Test de McNemar

Creación de tabla de contingencia o matriz de confusión.

In [10]:
from mlxtend.evaluate import mcnemar_table
table = mcnemar_table(y_test, y_pred_knn, y_pred_lda)
print(table)

[[41  2]
 [ 0  2]]


Aplicación del Test de McNemar con correción de continuidad: cálculo del estadístico χ² y el valor p.

In [11]:
from mlxtend.evaluate import mcnemar

chi2, p = mcnemar(table, corrected=True)
print('Estadístico chi-cuadrado:', chi2)
print('Valor p:', p)

Estadístico chi-cuadrado: 0.5
Valor p: 0.47950012218695337


Interpretación del resultado con un nivel de significatividad α = 0.05

---



In [12]:
alpha = 0.05
if p > alpha:
	print('Se acepta la hipótesis nula: ambos modelos tienen el mismo rendimiento.')
else:
	print('Se rechaza la hipótesis nula: los modelos no tienen el mismo rendimiento.')

Se acepta la hipótesis nula: ambos modelos tienen el mismo rendimiento.


# 5x2cv paired t test

Aplicación del test t de Student pareado sobre los resultados de aplicar 5x2-fold cross validation.

In [13]:
from mlxtend.evaluate import paired_ttest_5x2cv

t, p = paired_ttest_5x2cv(knn, lda, X, y, random_seed=1)

print('Estadístico t: ', t)
print('Valor p: ', p)

Estadístico t:  -1.3693063937629153
Valor p:  0.2292049735542175


Interpretación del resultado con un valor de significatividad α = 0.05


In [14]:
alpha = 0.05
if p > alpha:
	print('Se acepta la hipótesis nula: ambos modelos tienen el mismo rendimiento.')
else:
	print('Se rechaza la hipótesis nula: los modelos no tienen el mismo rendimiento.')

Se acepta la hipótesis nula: ambos modelos tienen el mismo rendimiento.
